In [ ]:
!pip install pytorch-tabnet


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.5/44.5 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 118.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 88.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 62.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 44.1 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstallin

In [ ]:
import numpy as np
import pandas as pd

from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

from imblearn.over_sampling import SMOTE
import xgboost as xgb
from pytorch_tabnet.tab_model import TabNetClassifier




df = pd.read_csv('/content/data.csv')
X_full = df.drop(columns=['ID', 'class'])
y_full = df['class'].astype('category').cat.codes

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
fold_acc = []

for fold, (tr, val) in enumerate(skf.split(X_full, y_full), 1):
    # ---- 1. RFE ----
    rfe = RFE(RandomForestClassifier(n_estimators=200, random_state=fold),
              n_features_to_select=80)
    X_tr, y_tr = X_full.iloc[tr], y_full.iloc[tr]
    X_val, y_val = X_full.iloc[val], y_full.iloc[val]

    X_tr = rfe.fit_transform(X_tr, y_tr)
    X_val = rfe.transform(X_val)

    # ---- 2. SMOTE + scale ----
    X_tr, y_tr = SMOTE(random_state=42).fit_resample(X_tr, y_tr)
    scaler = StandardScaler()
    X_tr = scaler.fit_transform(X_tr).astype(np.float32)
    X_val = scaler.transform(X_val).astype(np.float32)

    # ---- 3. TabNet ensemble----
    tabnet_preds = np.zeros(len(y_val))
    for seed in [0, 7, 21, 42, 99]:
        tab = TabNetClassifier(
            n_d=64, n_a=64, n_steps=7, gamma=1.75,
            lambda_sparse=1e-4, seed=seed,
            optimizer_params={"lr":0.01}
        )
        tab.fit(
            X_tr, y_tr,
            eval_set=[(X_val, y_val)], eval_metric=['accuracy'],
            max_epochs=500, patience=60,
            batch_size=32, virtual_batch_size=16
        )
        tabnet_preds += tab.predict_proba(X_val)[:,1]
    tabnet_preds /= 5

    # ---- 4. XGBoost ----
    xg = xgb.XGBClassifier(
        n_estimators=500, max_depth=5,
        learning_rate=0.03, subsample=0.85,
        colsample_bytree=0.85, reg_lambda=1.0,
        random_state=42, eval_metric='logloss'
    )
    xg.fit(X_tr, y_tr)
    xgb_preds = xg.predict_proba(X_val)[:,1]

    # ---- 5. Stacker ----
    blend_X = np.vstack([tabnet_preds, xgb_preds]).T
    stacker = LogisticRegression()
    stacker.fit(blend_X, y_val)
    final = (stacker.predict_proba(blend_X)[:,1] > 0.5).astype(int)

    acc = accuracy_score(y_val, final)
    fold_acc.append(acc)
    print(f"\nFold {fold} | ACC={acc:.3f}")
    print(confusion_matrix(y_val, final))
    print(classification_report(y_val, final, digits=3))

print(f"\nCV MEAN ACCURACY = {np.mean(fold_acc):.4f} ± {np.std(fold_acc):.4f}")


/usr/local/lib/python3.11/dist-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 1.6834  | val_0_accuracy: 0.42857 |  0:00:01s
epoch 1  | loss: 1.43713 | val_0_accuracy: 0.57143 |  0:00:01s
epoch 2  | loss: 1.5881  | val_0_accuracy: 0.57143 |  0:00:02s
epoch 3  | loss: 1.60518 | val_0_accuracy: 0.48571 |  0:00:02s
epoch 4  | loss: 1.58041 | val_0_accuracy: 0.62857 |  0:00:02s
epoch 5  | loss: 0.93878 | val_0_accuracy: 0.65714 |  0:00:02s
epoch 6  | loss: 1.18951 | val_0_accuracy: 0.65714 |  0:00:02s
epoch 7  | loss: 0.97196 | val_0_accuracy: 0.8     |  0:00:03s
epoch 8  | loss: 0.74142 | val_0_accuracy: 0.77143 |  0:00:03s
epoch 9  | loss: 0.68043 | val_0_accuracy: 0.68571 |  0:00:03s
epoch 10 | loss: 0.71604 | val_0_accuracy: 0.74286 |  0:00:04s
epoch 11 | loss: 0.96457 | val_0_accuracy: 0.77143 |  0:00:04s
epoch 12 | loss: 0.66198 | val_0_accuracy: 0.8     |  0:00:04s
epoch 13 | loss: 0.73069 | val_0_accuracy: 0.71429 |  0:00:04s
epoch 14 | loss: 0.85239 | val_0_accuracy: 0.68571 |  0:00:05s
epoch 15 | loss: 1.11505 | val_0_accuracy: 0.6     |  0

/usr/local/lib/python3.11/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
/usr/local/lib/python3.11/dist-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 1.7303  | val_0_accuracy: 0.51429 |  0:00:00s
epoch 1  | loss: 2.23468 | val_0_accuracy: 0.6     |  0:00:00s
epoch 2  | loss: 1.26778 | val_0_accuracy: 0.54286 |  0:00:00s
epoch 3  | loss: 1.5365  | val_0_accuracy: 0.51429 |  0:00:00s
epoch 4  | loss: 1.86514 | val_0_accuracy: 0.45714 |  0:00:01s
epoch 5  | loss: 1.46887 | val_0_accuracy: 0.6     |  0:00:01s
epoch 6  | loss: 1.26146 | val_0_accuracy: 0.48571 |  0:00:01s
epoch 7  | loss: 1.18812 | val_0_accuracy: 0.48571 |  0:00:01s
epoch 8  | loss: 1.25233 | val_0_accuracy: 0.54286 |  0:00:02s
epoch 9  | loss: 1.48023 | val_0_accuracy: 0.71429 |  0:00:02s
epoch 10 | loss: 1.14623 | val_0_accuracy: 0.68571 |  0:00:02s
epoch 11 | loss: 0.88668 | val_0_accuracy: 0.57143 |  0:00:02s
epoch 12 | loss: 0.84925 | val_0_accuracy: 0.48571 |  0:00:02s
epoch 13 | loss: 0.76738 | val_0_accuracy: 0.6     |  0:00:03s
epoch 14 | loss: 0.82822 | val_0_accuracy: 0.6     |  0:00:03s
epoch 15 | loss: 0.98727 | val_0_accuracy: 0.68571 |  0

/usr/local/lib/python3.11/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
/usr/local/lib/python3.11/dist-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")


...

epoch 0  | loss: 1.61419 | val_0_accuracy: 0.55882 |  0:00:00s
epoch 1  | loss: 2.63963 | val_0_accuracy: 0.52941 |  0:00:00s
epoch 2  | loss: 2.66373 | val_0_accuracy: 0.58824 |  0:00:00s
epoch 3  | loss: 1.99362 | val_0_accuracy: 0.61765 |  0:00:01s
epoch 4  | loss: 1.4307  | val_0_accuracy: 0.61765 |  0:00:01s
epoch 5  | loss: 1.77454 | val_0_accuracy: 0.55882 |  0:00:01s
epoch 6  | loss: 1.79756 | val_0_accuracy: 0.70588 |  0:00:01s
epoch 7  | loss: 1.52033 | val_0_accuracy: 0.67647 |  0:00:02s
epoch 8  | loss: 1.353   | val_0_accuracy: 0.76471 |  0:00:02s
epoch 9  | loss: 1.02825 | val_0_accuracy: 0.76471 |  0:00:02s
epoch 10 | loss: 1.00588 | val_0_accuracy: 0.76471 |  0:00:02s
epoch 11 | loss: 0.74657 | val_0_accuracy: 0.73529 |  0:00:02s
epoch 12 | loss: 0.85342 | val_0_accuracy: 0.73529 |  0:00:03s
epoch 13 | loss: 0.84646 | val_0_accuracy: 0.82353 |  0:00:03s
epoch 14 | loss: 0.81168 | val_0_accuracy: 0.79412 |  0:00:03s
epoch 15 | loss: 0.7963  | val_0_accuracy: 0.58824 |  0

/usr/local/lib/python3.11/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
/usr/local/lib/python3.11/dist-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 1.53808 | val_0_accuracy: 0.41176 |  0:00:00s
epoch 1  | loss: 1.22156 | val_0_accuracy: 0.55882 |  0:00:00s
epoch 2  | loss: 1.56242 | val_0_accuracy: 0.5     |  0:00:00s
epoch 3  | loss: 1.4261  | val_0_accuracy: 0.61765 |  0:00:00s
epoch 4  | loss: 1.0943  | val_0_accuracy: 0.55882 |  0:00:01s
epoch 5  | loss: 0.96801 | val_0_accuracy: 0.61765 |  0:00:01s
epoch 6  | loss: 1.05824 | val_0_accuracy: 0.55882 |  0:00:01s
epoch 7  | loss: 1.37942 | val_0_accuracy: 0.55882 |  0:00:01s
epoch 8  | loss: 0.8905  | val_0_accuracy: 0.61765 |  0:00:02s
epoch 9  | loss: 0.94703 | val_0_accuracy: 0.55882 |  0:00:02s
epoch 10 | loss: 0.72335 | val_0_accuracy: 0.52941 |  0:00:02s
epoch 11 | loss: 0.80163 | val_0_accuracy: 0.5     |  0:00:03s
epoch 12 | loss: 0.92837 | val_0_accuracy: 0.52941 |  0:00:03s
epoch 13 | loss: 0.47882 | val_0_accuracy: 0.61765 |  0:00:03s
epoch 14 | loss: 1.34261 | val_0_accuracy: 0.70588 |  0:00:04s
epoch 15 | loss: 1.67923 | val_0_accuracy: 0.61765 |  0

/usr/local/lib/python3.11/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)



Fold 5 | ACC=0.882
[[15  2]
 [ 2 15]]
              precision    recall  f1-score   support

           0      0.882     0.882     0.882        17
           1      0.882     0.882     0.882        17

    accuracy                          0.882        34
   macro avg      0.882     0.882     0.882        34
weighted avg      0.882     0.882     0.882        34


CV MEAN ACCURACY = 0.9022 ± 0.0501
